In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import tensorflow as tf
import cudf
import cuml
import cupy
import matplotlib.pyplot as plt

In [5]:
data_path = '../input/h-and-m-personalized-fashion-recommendations/'
image_path = '../input/h-and-m-personalized-fashion-recommendations/images/'

In [6]:
df = pd.read_csv(data_path+'transactions_train.csv', dtype={'article_id': str, 't_dat': object, 'customer_id': str, 'price': float, 'sales_channel_id': int})
df.info()

 - Pre-processing

In [7]:
df['t_dat'] = pd.to_datetime(df['t_dat'])
df['timestamp'] = pd.to_datetime(df['t_dat'], unit='ms').astype(np.int64)//1e9
df['customer_id'] = df['customer_id'].astype('str')

In [8]:
df.sample(5)

In [9]:
null_cnt = df.isna().sum()
null_cnt

In [10]:
#Unique items in each field
total_transactions = len(df)
for col in ['customer_id', 'article_id']:
    uq = len(df[col].unique())
    print('Unique in', col, uq, 'Density:', total_transactions/uq)

In [11]:
uniq_customer = list(df['customer_id'].unique())
uniq_article = list(df['article_id'].unique())
print(len(uniq_customer), len(uniq_article))

In [12]:
uniq_customer[:3]

In [13]:
sub_df = df.sample(90_000, random_state=26)
sub_df

In [22]:
itemSeq = sub_df.groupby('customer_id')['article_id'].apply(lambda x: x.to_string(index=False, header=False))
itemSeq

In [30]:
orderedItems = []
customers = []
items = []
for key, seq in itemSeq.iteritems():
    s = seq.split('\n')
#     print(key)
    customers.append(key)
    items.append(' '.join(s))
    if len(s) > 1:
        orderedItems.append(' '.join(s))
orderedItems

In [31]:
final_df = pd.DataFrame({'customer_id':customers, 'order_seq': items})
final_df

In [32]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(orderedItems)

In [33]:
total_items = len(tokenizer.word_index) + 1
total_items

In [34]:
input_sequences = []
for line in orderedItems:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [35]:
max_seq_len = max([len(inp) for inp in input_sequences])
max_seq_len

In [36]:
seq = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_seq_len-1, padding='pre')

In [37]:
seq[0]

In [38]:
X = seq[:,:-1]
labels = seq[:,-1]
# One-hot encode with keras convert list to a categorical. The number of classes which is my number of words.
Y = tf.keras.utils.to_categorical(labels, num_classes=total_items)


In [39]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(total_items, 100, input_length=max_seq_len-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)))
model.add(tf.keras.layers.Dense(total_items, activation='softmax'))
adam = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(X, Y, epochs=25, verbose=1)
print(model)

In [47]:
def findNext12(seed_text):
    next_items = 12
    #Prediction
    print(seed_text)
    ans = ""
    for _ in range(next_items):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0)[0])
#         print(predicted)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        ans += " " + output_word
    return ans[1:]

In [48]:
findNext12('0589544003 0714828001')

In [ ]:
final_df['prediction'] = final_df['order_seq'].apply(lambda x: findNext12(x))
final_df